<h1 style="text-align: center; color: red; font-size: 25pt; font-weight: bold; text-shadow: 1px 1px 1px rgb(50, 100, 100)"> Terror Attacks In Nigeria .... </h1> 
<p style="font-size: 12pt; text-align: center"> Abubakar Abdulkadir</p>

<img src="images/bg.jpg" style="width: 100%; height:350px; display: inline-block" />

<h1 style='background-color: red; padding: 10px; color: white'> 1.0 Data Preprocessing </h1>

Data preprocessing is the process of preparing raw data for analysis by cleaning, transforming, and integrating it into a suitable format. The goal of data preprocessing phase is to improve the quality and efficiency of data analysis by removing irrelevant data, filling missing values, and converting data into a more appropriate format.

The process typically involves several steps, including 
- data cleaning 
- data integration 
- data transformation
- and data reduction.

Effective data preprocessing is essential for accurate and efficient data mining. By improving the quality and consistency of the data, data preprocessing can help data analysts to identify patterns and trends, make better decisions, and gain deeper insights into the underlying data.

In [47]:
# import libraries
import pandas as pd
import numpy as np
import re
import warnings


# Suppress warning messages from Pandas
warnings.filterwarnings('ignore')


# importing the dataset
df_acled = pd.read_csv('datasets/cleaned/df_acled_nig.csv') # ACLED dataset 
df_gtd = pd.read_csv('datasets/cleaned/df_gtd_nig.csv') # GTD dataset 1970 to 2020
df_lga = pd.read_csv('datasets/cleaned/df_lga_nig.csv') # LGA datasets
df_lga_features = pd.read_csv('datasets/cleaned/df_lga_features.csv') # LGA datasets

## 1.0 All functions used for data cleaning

All the functions used for this stage of the project is defined in this section. Most of the function are used for the cleaning of the LGA features dataset because they contain the most level of dirt.

In [153]:
# This function extracts numbers from a dirty string. It will match numbers like '1', '20', '01', '10,10', '10,20.00' etc
def get_numbers_without_units(row, column, no_match_value='default'):
    perc = row[column]
    try:
        perc = re.findall('\d+(?:,?\d*)*\.*\d*', perc)[0]
        perc = perc.replace(',', '')
    except:
        if no_match_value == 'default':
            return perc
        else:
            return no_match_value
    return perc


# This function returns only sequential alphabets in a string. It matches only strings strating with aphabets
def retrieve_alphabets(row, column_name):
    res = row[column_name]
    try:
        res = ' '.join(re.findall('[a-zA-Z]+', res))
    except:
        return ''
    return res.lower()


def replace_word(row, word, repl_word, column):
    '''
        searches for word in column. If it exists, it replaces the column with repl_word
    '''
    try:
        if word.lower() in row[column].lower(): 
            return  repl_word.lower()

        return row[column].lower()
    except:
        return ''
    
    return ''
 

def replace_words(dataframe, repls, column):
    '''
        Searches for a string B in A. If B is in A it replaces A with another string C.
        
        Arguments:
        -----------------------------
        row - single row in a dataframe
        column - Name of column in the row which contains string A
        repls - list of tupples i.e [('B', 'C'), ('B', 'C')]
    '''
    for repl in repls:
        dataframe[column] = dataframe.apply(replace_word, word=repl[0], repl_word=repl[1], column=column, axis=1)
        
    return dataframe[column]


def binarize(row, word, column):
    '''
        Searches for a word in a string. If found, it replaces the entire string with 1 else it replaces it with 0
    '''
    row_word = row[column]
    try:
        if word.lower() in row_word.lower():
            return 1
        else:
            return 0
    except:
        return 0
    
    
def extract_numbers_units(row, values, column, find_all=False):
    '''
        Extracts values and their units from a string.
    '''
    cur_val = row[column]

    try:
        for value in values:
            res = re.findall('\d+(?:,?\d*)*\.*\s*{}'.format(value), cur_val)
            if res:
                break

        if res:
            return res[0]
        else:
            return cur_val
    except:
        return ''
    

def convert_values(row, values, column):
    '''
        converts values with different units to a single unit
        
        parameters:
        ---------------------------
        values - list of tupples - first entry in the tupple is the unit and the second entry is the convertion rate. 
                i.e [('m', 0.75), ('hrs', 60)]
        
    '''
    cur_val = row[column]
    for value in values:
        if value[0] in cur_val:
            res = re.search(r'\d+(?:,?\d*)*\.*', cur_val)
            if res:
                res = res.group().replace(',', '')
                res = float(res) * value[1]
                return res
    return cur_val.replace(',', '')


def extract_convert_curr(row, column, currencies):
    '''
        extracting amount with the currency sign in a string, then converts the amount with the value
        of the currency attached
    '''
    cur_val = row[column]
    try:
        for currency in currencies:
            pattern = fr'{re.escape(currency[0])}\s?\d+(,\d+)*(\.\d+)?\d*'
            res = re.search(pattern, cur_val)

            if res:
                res = res.group()
                res = re.search(r'\d+(,\d+)?(\.\d+)?\d?', res).group()
                res = res.replace(',', '')
                return float(res) * currency[1]


        res = re.search(r'\d+(,\d+)?(\.\d+)?\d?', cur_val)
        if res:
            res =  res.group().replace(',', '')
            return float(res)
    except:
        return ''

    return ''
    
def extract_age_range(row, column):
    cur_val = row[column]
    try:
        res = re.search(r'\d+\-?(\d+)?', cur_val)

        if res:
            res = res.group()
            return res
        if 'youth' in cur_val.lower():
            return '15-35'
        elif 'young adult' in cur_val.lower():
            return '18-35'
        elif 'middle' in cur_val.lower():
            return '65-40'
    except:
        return ''
    return ''

## 1.1 Data Cleaning

Data cleaning involves removing noise, errors, and inconsistencies from the data. 

### 1.1.1 Cleaning the ACLED Dataset

#### Handling Null values

In [48]:
df_acled.isnull().sum()

event_id_cnty             0
event_date                0
year                      0
time_precision            0
disorder_type             0
event_type                0
sub_event_type            0
actor1                    0
assoc_actor_1         20021
inter1                    0
actor2                 6854
assoc_actor_2         21632
inter2                    0
interaction               0
civilian_targeting    17231
iso                       0
region                    0
country                   0
admin1                    1
admin2                    1
admin3                28160
location                  0
latitude                  0
longitude                 0
geo_precision             0
source                    0
source_scale              0
notes                     0
fatalities                0
tags                  23316
timestamp                 0
dtype: int64

In [49]:
# Percentage of null values in fields containing nulls
null_pct = df_acled.isnull().mean() * 100
null_pct = null_pct[null_pct.values > 0]
null_pct

assoc_actor_1          71.097301
actor2                 24.339489
assoc_actor_2          76.818182
civilian_targeting     61.189631
admin1                  0.003551
admin2                  0.003551
admin3                100.000000
tags                   82.798295
dtype: float64

8 out of the available 32 features contain null values. Of these eight, their percentages show that majority of them have above 70% null entries. For features with percentages above the 50% benchmark, dropping of the features are considered. Hence, we will consider inputation for actor2, admin1 and admin2. 

- The actor2 feature represents the secondary actor involved in the event, which can be another armed group, an international organization, a civilian group, or other types of actors. Null values in this field could mean that there is no record for this actor. For the purpose of this project, imputation by mode will undesirable because it could lead to misinformation or a potential unreliability of information derived from this field. Hence, null values in this field are replaced with a constant keyword 'unknown'. 

- The admin1 feature represents the highest level of administrative division in the location, such as a state, province, or region, depending on the country. The admin2 feature represents the second-highest level of administrative division, such as a district, county, or municipality. Hence, like the actor2, implacing nulls with constant keyword "unkown" is prefered.

In [50]:
# drop all columns where percentage of nulls are greater than 50%
drop_cols = null_pct[null_pct > 50].index.tolist()
df_acled = df_acled.drop(columns=drop_cols)

# input 'unknown' keyword for actor2, admin1, admin2 features
df_acled[['actor2', 'admin1', 'admin2']] = df_acled[['actor2', 'admin1', 'admin2']].fillna('unknown')

In [51]:
df_acled.isnull().sum()

event_id_cnty     0
event_date        0
year              0
time_precision    0
disorder_type     0
event_type        0
sub_event_type    0
actor1            0
inter1            0
actor2            0
inter2            0
interaction       0
iso               0
region            0
country           0
admin1            0
admin2            0
location          0
latitude          0
longitude         0
geo_precision     0
source            0
source_scale      0
notes             0
fatalities        0
timestamp         0
dtype: int64

#### Handling Inconsistencies

In [52]:
# select all the categorical features and describing them
df_acled.select_dtypes(include=['object']).describe()

,event_id_cnty,event_date,disorder_type,event_type,sub_event_type,actor1,actor2,region,country,admin1,admin2,location,source,source_scale,notes
count,28160,28160,28160,28160,28160,28160,28160,28160,28160,28160,28160,28160,28160,28160,28160
unique,28160,4489,4,6,24,667,684,1,1,38,745,4261,3119,21,26640
top,NIG30571,18 March 2023,Political violence,Violence against civilians,Attack,Protesters (Nigeria),Civilians (Nigeria),Western Africa,Nigeria,Borno,Abuja Municipal,Abuja,Vanguard (Nigeria),National,"Organised labour, including the NLC and its ci..."
freq,1,64,19523,9473,7522,6162,11290,28160,28160,4527,1223,1055,3237,19514,24


In [53]:
df_acled.select_dtypes(include=['object']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28160 entries, 0 to 28159
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   event_id_cnty   28160 non-null  object
 1   event_date      28160 non-null  object
 2   disorder_type   28160 non-null  object
 3   event_type      28160 non-null  object
 4   sub_event_type  28160 non-null  object
 5   actor1          28160 non-null  object
 6   actor2          28160 non-null  object
 7   region          28160 non-null  object
 8   country         28160 non-null  object
 9   admin1          28160 non-null  object
 10  admin2          28160 non-null  object
 11  location        28160 non-null  object
 12  source          28160 non-null  object
 13  source_scale    28160 non-null  object
 14  notes           28160 non-null  object
dtypes: object(15)
memory usage: 3.2+ MB


The only inconsistency in the categorical subset of the dataset in terms of datatype is the event_date which is an object datatype instaed of datetime. Although, region and country have just one unique value each; western africa and Nigeria. There is no much information to be gained from those features hence, we can do away with them.

In [54]:
# drop the region and country feature
df_acled = df_acled.drop(columns=['region', 'country'])

In [55]:
# convert the event_date to datetime feature
df_acled['event_date'] = pd.to_datetime(df_acled['event_date'], format='%d %B %Y')

In [56]:

df_acled.select_dtypes(include=['float64', 'int64', 'float32', 'int32']).describe()

,year,time_precision,inter1,inter2,interaction,iso,latitude,longitude,geo_precision,fatalities,timestamp
count,28160.000000,28160.000000,28160.000000,28160.000000,28160.000000,28160.0,28160.000000,28160.000000,28160.000000,28160.000000,2.816000e+04
mean,2018.499219,1.112109,3.613033,3.681214,36.898366,566.0,8.797115,8.038439,1.362393,3.230256,1.620157e+09
std,3.410668,0.329815,1.747718,2.995227,17.798928,0.0,2.570994,3.022455,0.515148,13.304044,3.676026e+07
min,2009.000000,1.000000,1.000000,0.000000,10.000000,566.0,2.855900,1.328200,1.000000,0.000000,1.552576e+09
25%,2016.000000,1.000000,2.000000,1.000000,16.000000,566.0,6.453100,6.199800,1.000000,0.000000,1.591052e+09
50%,2020.000000,1.000000,4.000000,3.000000,37.000000,566.0,8.883300,7.424300,1.000000,0.000000,1.618566e+09
75%,2021.000000,1.000000,5.000000,7.000000,55.000000,566.0,11.176700,9.284600,2.000000,2.000000,1.652117e+09
max,2023.000000,3.000000,8.000000,8.000000,78.000000,566.0,13.784400,14.650000,3.000000,600.000000,1.679960e+09


There is no so much inconsistency in the numerical data. Aside that iso contain a single unique value. Although, we cannot drop such column because they might be useful in making converions

### 1.1.2 Cleaning the GTD Dataset

#### Handling Null values 

In [57]:
# Percentage of null values in fields containing nulls
null_pct = df_gtd.isnull().mean() * 100
null_pct = null_pct[null_pct.values > 0]
null_pct

approxdate    94.764922
resolution    96.165388
latitude       0.800267
longitude      0.800267
location      53.384461
                ...    
addnotes      63.837946
scite1         1.000333
scite2        24.524842
scite3        46.498833
related       66.305435
Length: 99, dtype: float64

In [58]:
null_pct[null_pct.values > 60]

approxdate            94.764922
resolution            96.165388
alternative           87.862621
alternative_txt       87.862621
attacktype2           74.774925
                        ...    
hostkidoutcome        85.111704
hostkidoutcome_txt    85.111704
nreleased             85.495165
addnotes              63.837946
related               66.305435
Length: 71, dtype: float64

Out of the 135 features of the dataset, 99 of them contain null values. 71 out of the 99 contain null values above 60% of the total values. To reduce the features of this dataset, we consider droping features with nulls above 60% as imputation will lead to so much artificiality in the dataset. Hence, there is a need to decide on means to handle the missing values is the remaing 18 features.

In [59]:
null_pct[null_pct.values < 60]

latitude             0.800267
longitude            0.800267
location            53.384461
summary              1.000333
targsubtype1         2.784261
targsubtype1_txt     2.784261
corp1                2.834278
target1              0.083361
natlty1              0.166722
natlty1_txt          0.166722
nperps               3.251084
nperpcap             1.733911
claimed              1.000333
weapsubtype1         7.819273
weapsubtype1_txt     7.819273
nkill                9.286429
nkillus              1.067022
nkillter             5.701901
nwound              28.292764
nwoundus             1.117039
nwoundte             9.153051
propextent          52.867623
propextent_txt      52.867623
propvalue           56.202067
propcomment         47.765922
scite1               1.000333
scite2              24.524842
scite3              46.498833
dtype: float64

For these fields where the null percentage are less than 60%, we will consider replacing null values with the keyword 'unkown' for consistency with the acled dataset.

In [60]:
# drop all columns where percentage of nulls are greater than 60%
drop_cols = null_pct[null_pct > 60].index.tolist()
df_gtd.drop(columns=drop_cols, inplace=True)

# input 'unknown' keyword for actor2, admin1, admin2 features
repl_cols = null_pct[null_pct < 60].index.tolist()
df_gtd[repl_cols] = df_gtd[repl_cols].fillna('unknown')

In [61]:
df_gtd.isnull().sum()

eventid     0
iyear       0
imonth      0
iday        0
extended    0
           ..
dbsource    0
INT_LOG     0
INT_IDEO    0
INT_MISC    0
INT_ANY     0
Length: 64, dtype: int64

#### Handling inconsistencies

In [62]:
df_gtd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 64 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   eventid           5998 non-null   int64  
 1   iyear             5998 non-null   int64  
 2   imonth            5998 non-null   int64  
 3   iday              5998 non-null   int64  
 4   extended          5998 non-null   int64  
 5   country           5998 non-null   int64  
 6   country_txt       5998 non-null   object 
 7   region            5998 non-null   int64  
 8   region_txt        5998 non-null   object 
 9   provstate         5998 non-null   object 
 10  city              5998 non-null   object 
 11  latitude          5998 non-null   object 
 12  longitude         5998 non-null   object 
 13  specificity       5998 non-null   float64
 14  vicinity          5998 non-null   int64  
 15  location          5998 non-null   object 
 16  summary           5998 non-null   object 


In [63]:
# describe the first ten features
df_gtd.select_dtypes(include=['object']).iloc[0:, 0:12].describe()

,country_txt,region_txt,provstate,city,latitude,longitude,location,summary,attacktype1_txt,targtype1_txt,targsubtype1,targsubtype1_txt
count,5998,5998,5998,5998,5998.000000,5998.000000,5998,5998,5998,5998,5998.0,5998
unique,1,1,38,2615,3409.000000,3384.000000,1897,5809,9,19,92.0,92
top,Nigeria,Sub-Saharan Africa,Borno,Maiduguri,11.840793,13.141391,unknown,unknown,Armed Assault,Private Citizens & Property,75.0,Village/City/Town/Suburb
freq,5998,5998,2113,494,387.000000,387.000000,3202,60,2821,2914,1564.0,1564


In [64]:
# describe the next ten features
df_gtd.select_dtypes(include=['object']).iloc[0:,12:24].describe()

,corp1,target1,natlty1,natlty1_txt,gname,nperps,nperpcap,claimed,weaptype1_txt,weapsubtype1,weapsubtype1_txt,nkill
count,5998,5998,5998.0,5998,5998,5998.0,5998.0,5998.0,5998,5998.0,5998,5998.0
unique,2279,1947,37.0,37,75,50.0,24.0,4.0,7,27.0,27,91.0
top,Not Applicable,Village,147.0,Nigeria,Boko Haram,-99.0,0.0,0.0,Firearms,5.0,Unknown Gun Type,0.0
freq,939,1284,5789.0,5789,2901,4655.0,5687.0,5297.0,3449,3034.0,3034,1421.0


In [65]:
# describe the last features
df_gtd.select_dtypes(include=['object']).iloc[0:,24:].describe()

,nkillus,nkillter,nwound,nwoundus,nwoundte,propextent,propextent_txt,propvalue,propcomment,scite1,scite2,scite3,dbsource
count,5998.0,5998.0,5998.0,5998.0,5998.0,5998,5998,5998,5998,5998,5998,5998,5998
unique,3.0,53.0,73.0,4.0,9.0,5,5,14,1301,4181,3422,2470,7
top,0.0,0.0,0.0,0.0,0.0,unknown,unknown,unknown,unknown,unknown,unknown,unknown,START Primary Collection
freq,5933.0,4772.0,2745.0,5929.0,5415.0,3171,3171,3371,2865,60,1471,2789,5472


A look at the description of the dataset indicates that with the _txt surfix are categorical feature. For each of these features, there is a corresponding feature containing numerical values. Both the fields with categorical and numerical values convey same information, only that they have been handled differently. Hence, to reduce the cumbersomess of the dataset, we drop such numerical features which has corresponding categorical feature. This will allow the database categorical features to be used in different ways without data repetition.

List of these numerical features having corresponding categorical features
- natlty1
- propextent
- targsubtype1
- weapsubtype1

In [66]:
df_gtd.drop(['natlty1', 'propextent', 'targsubtype1', 'weapsubtype1'], inplace=True, axis=1)
df_gtd.shape

(5998, 60)

In [67]:
# select the first 12 numerical features from the gtd dataset and describe 
df_gtd.select_dtypes(include=['float64', 'int64', 'float32', 'int32']).iloc[0:, 0:12].describe()

,eventid,iyear,imonth,iday,extended,country,region,specificity,vicinity,crit1,crit2,crit3
count,5.998000e+03,5998.000000,5998.000000,5998.000000,5998.000000,5998.0,5998.0,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000
mean,2.015489e+11,2015.427643,5.981327,15.623374,0.151884,147.0,11.0,1.536345,0.059687,0.977993,0.997833,0.915805
std,4.245149e+08,4.248259,3.418210,8.840217,0.358938,0.0,0.0,0.859825,0.236925,0.146719,0.046509,0.277703
min,1.976021e+11,1976.000000,1.000000,1.000000,0.000000,147.0,11.0,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.013112e+11,2013.000000,3.000000,8.000000,0.000000,147.0,11.0,1.000000,0.000000,1.000000,1.000000,1.000000
50%,2.016031e+11,2016.000000,6.000000,15.000000,0.000000,147.0,11.0,1.000000,0.000000,1.000000,1.000000,1.000000
75%,2.018120e+11,2018.000000,9.000000,23.000000,0.000000,147.0,11.0,2.000000,0.000000,1.000000,1.000000,1.000000
max,2.021063e+11,2021.000000,12.000000,31.000000,1.000000,147.0,11.0,5.000000,1.000000,1.000000,1.000000,1.000000


In [68]:
# select the first last set of numerical features from the gtd dataset and describe 
df_gtd.select_dtypes(include=['float64', 'int64', 'float32', 'int32']).iloc[0:, 12:].describe()

,doubtterr,multiple,success,suicide,attacktype1,targtype1,guncertain1,individual,weaptype1,property,ishostkid,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
count,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000,5998.0,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000
mean,0.106369,0.336946,0.915972,0.080360,3.376292,10.536512,0.168223,0.0,6.125208,-0.819940,-0.154051,-4.049350,-4.029343,0.033011,-3.855952
std,0.495122,0.472706,0.277453,0.271873,2.111835,5.940474,0.374095,0.0,2.164526,3.385112,1.689291,4.479985,4.500215,0.277397,4.498004
min,-9.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.0,2.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000
25%,0.000000,0.000000,1.000000,0.000000,2.000000,4.000000,0.000000,0.0,5.000000,0.000000,0.000000,-9.000000,-9.000000,0.000000,-9.000000
50%,0.000000,0.000000,1.000000,0.000000,2.000000,14.000000,0.000000,0.0,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,5.000000,14.000000,0.000000,0.0,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,9.000000,22.000000,1.000000,0.0,13.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


There are no observable inconsistency from this pheriperal outlook on the dataset. Hence we will pass the numerical features as they are. 

### 1.1.3 Cleaning the LGA Dataset

#### Handling Nulls

In [69]:
# Percentage of null values in fields containing nulls
null_pct = df_lga.isnull().mean() * 100
null_pct = null_pct[null_pct.values > 0]
null_pct

state_code    1.679587
dtype: float64

Only the statecode feature is containing null. That feature is supposed tobe distinct. Hence, we can not input them with a measure of central tendency. We retain the tradition for this project and replace null with 'unknown'.  

In [70]:
# input 'unknown' for null values
df_lga.fillna('unknown', inplace=True)

In [71]:
df_lga.isnull().sum()

id              0
name            0
state_id        0
state_code      0
state_name      0
country_id      0
country_code    0
country_name    0
latitude        0
longitude       0
wikiDataId      0
dtype: int64

#### Checking for inconsistencies

In [72]:
df_lga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774 entries, 0 to 773
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            774 non-null    int64  
 1   name          774 non-null    object 
 2   state_id      774 non-null    int64  
 3   state_code    774 non-null    object 
 4   state_name    774 non-null    object 
 5   country_id    774 non-null    int64  
 6   country_code  774 non-null    object 
 7   country_name  774 non-null    object 
 8   latitude      774 non-null    float64
 9   longitude     774 non-null    float64
 10  wikiDataId    774 non-null    object 
dtypes: float64(2), int64(3), object(6)
memory usage: 66.6+ KB


### 1.1.4 Cleaning the LGA Feature Dataset

Because this dataset was collected from ChatGPT API's unstructured response, it contains the most dirt and inonsistencies. Unlike the other datasets, which must have been precleaned by their custodians, this dataset is converted into pandas dataframe from chatGPT API's raw responses. Sometimes, for categorical variables, it uses different synonymns which means almost same thing to quantify or qualify different local government. Hence, this dataset is needing preprocessing for each of the feature collected and careful observation of the values in each feature.

<b><i>NB: Functions used for the cleaning of this dataset are organised at the bottom of this notebook under the cleaning functions subheading</i></b>

In [73]:
df_lga_features.iloc[0:, 0:11].head()

,Lga Name,percentage of educated,Level of Isolation,Natural Barriers,Elevation,Population,Major Ocupation,Political stability,Economic Situation,Average Cost of living,Employment rate
0,Aba North,70%,Low,None,104m (341ft),"382,784",Trading and Commerce,Relatively Stable,Struggling,"â‚¦120,000 ($312) per month",65%
1,Aba South,70%,Low,None,30m,1 million,Trading,Relatively stable,Struggling,"â‚¦50,000",45%
2,Arochukwu,70%,Low,Hills and Rivers,1500ft,"150,000",Agriculture,Relatively stable,Developing,"â‚¦40,000",60%
3,Bende,90%,Low,Rivers,200 meters,"100,000",Farming,Stable,Struggling,"â‚¦50,000",70%
4,Ikwuano,70%,Low,Hills and Rivers,190m,"150,000",Farming,Relatively Stable,Struggling,"â‚¦50,000 ($125)",40%


A look at the first 10 columns of this dataset affirms the previous conviction that almost all the features of this dataset will be needing cleaning. Most of the features contain different units and weird characters.

#### 1.1.4.1 Cleaning the Percentage Educated features

In [74]:
# checking the distinct values in the dataset
df_lga_features['percentage of educated'].unique()

array(['70%', '90%', '65%', '60%;', '85%', '65%,', '75%,', '75%', '72%',
       '67%', '80%', '30%', '70%,', '50%', '60%', '68%', '64%', '58%',
       '85%,', '78%', '68%,', '85%;', '74%', '51%', '73%', '84%', '47%',
       '72%,', '60%,', '95%', '70', '62%', '63%', '80%,', '69%', '75%;',
       '81%', '57%', '80', '70%;', '40%', '42%', '76%', '45%', '55%',
       '75% |', '80%;', '54%', nan, '82%,', '71%', '65%;', '40%,', '83%',
       '79%,', '75', '56%', '50%,', '82%', '65%.', '87%', '90%,', '81%,',
       '40%.'], dtype=object)

In [75]:
# apply function to extract only the digits in the value as float
df_lga_features['percentage of educated'] = df_lga_features.apply(get_numbers_without_units
                                                                , column='percentage of educated'
                                                                , no_match_value=''
                                                                , axis=1)
df_lga_features['percentage of educated'].unique()

array([70.0, 90.0, 65.0, 60.0, 85.0, 75.0, 72.0, 67.0, 80.0, 30.0, 50.0,
       68.0, 64.0, 58.0, 78.0, 74.0, 51.0, 73.0, 84.0, 47.0, 95.0, 62.0,
       63.0, 69.0, 81.0, 57.0, 40.0, 42.0, 76.0, 45.0, 55.0, 54.0, '',
       82.0, 71.0, 83.0, 79.0, 56.0, 87.0], dtype=object)

#### 1.1.4.2 Cleaning the Level of Isolation features

In [76]:
# check values in level of isolation
df_lga_features['Level of Isolation'].unique()

array(['Low', 'Low;', 'Low,', 'Moderate', 'low', 'High', 'Medium',
       'Minimal', 'Moderate,', 'low;', 'Low |', nan, 'moderate;', 'High,',
       'Moderate.', 'Moderate;', 'Low.', 'Medium,'], dtype=object)

In [77]:
# We need only the sequential alphabets from this feature and they should be lower case
df_lga_features['Level of Isolation'] = df_lga_features.apply(retrieve_alphabets, column_name="Level of Isolation", axis=1)
df_lga_features['Level of Isolation'].unique() 

array(['low', 'moderate', 'high', 'medium', 'minimal', ''], dtype=object)

In [78]:
# replace medium and minimal with moderate
words_list = [('medium', 'moderate'), ('minimal', 'moderate')]
df_lga_features['Level of Isolation'] = replace_words(df_lga_features, repls=words_list, column='Level of Isolation') 
df_lga_features['Level of Isolation'].unique() 

array(['low', 'moderate', 'high', ''], dtype=object)

#### 1.1.4.3 Cleaning the Natural Barriers features

In [79]:
# check values in Natural Barriers
df_lga_features['Natural Barriers'].unique()

array(['None', 'Hills and Rivers', 'Rivers', 'Rivers, Hills', 'Hills',
       'Hills;', 'Hills and Forests', 'Rivers,', 'River',
       'Hills and Plateaus', 'Mountainous', 'Mountains, Rivers',
       'Mountains,', 'Mountains', 'mountains', 'Mountains and Rivers',
       'None,', 'Rivers and Hills', 'High', 'Rivers, mountains',
       'Rivers and Mountains', 'River Niger', 'Few', 'The Niger River',
       'Rivers and creeks', 'Mountainous terrain', 'Hills/mountains,',
       'Rivers, Swamps', 'Rivers and Creeks', 'Rivers, hills',
       'Coastline, Rivers', 'Rivers and swamps', 'rivers and swamps;',
       'Mountains;', 'Mountains, Rivers, Forests', 'Hills and forests',
       'Yes', 'Hills, Rivers', 'Rivers |', 'Rivers and mountains',
       'Riverine;', 'Rivers and creeks,', 'Rivers and hills', 'Rivers;',
       'Deltaic plain', 'Hills and mountains', nan, 'Hills and rivers',
       'Rivers and hills;', 'Hills and Mountains',
       'Mountains, Forests, Rivers', 'Hilly', 'Hilly terra

This feature has so much different different values. But a look at the values indicate that many of the values can be classed into one. For an instance, we can have rivereine, rivers, river, etc. The field is not atomic too. We could consider replacing all similar values with a representative value. Although, for this purpose we will create new features for each distinct value.

Hence, we will classify this values into 
- River - Swamp  - Hill - Forest  - Mountain - ocean - valley - desert - rock - plateau - creeks - none

In [80]:
# define the categories for the different kinds of bareers
categories = ['River', 'Swamp', 'Hill', 'Forest', 'Mountain', 
              'ocean', 'valley', 'desert', 'rock', 'plateau', 
              'creeks', 'none']

# create features for each of the unique categories 
for category in categories:
    column_head = "Has " + category
    df_lga_features[column_head] = df_lga_features.apply(binarize, word=category, column='Natural Barriers', axis= 1)

#drop the natural barriers feature
df_lga_features.drop(columns='Natural Barriers', inplace=True)

df_lga_features.head()

,Lga Name,percentage of educated,Level of Isolation,Elevation,Population,Major Ocupation,Political stability,Economic Situation,Average Cost of living,Employment rate,...,Has Hill,Has Forest,Has Mountain,Has ocean,Has valley,Has desert,Has rock,Has plateau,Has creeks,Has none
0,Aba North,70.0,low,104m (341ft),"382,784",Trading and Commerce,Relatively Stable,Struggling,"â‚¦120,000 ($312) per month",65%,...,0,0,0,0,0,0,0,0,0,1
1,Aba South,70.0,low,30m,1 million,Trading,Relatively stable,Struggling,"â‚¦50,000",45%,...,0,0,0,0,0,0,0,0,0,1
2,Arochukwu,70.0,low,1500ft,"150,000",Agriculture,Relatively stable,Developing,"â‚¦40,000",60%,...,1,0,0,0,0,0,0,0,0,0
3,Bende,90.0,low,200 meters,"100,000",Farming,Stable,Struggling,"â‚¦50,000",70%,...,0,0,0,0,0,0,0,0,0,0
4,Ikwuano,70.0,low,190m,"150,000",Farming,Relatively Stable,Struggling,"â‚¦50,000 ($125)",40%,...,1,0,0,0,0,0,0,0,0,0


#### 1.1.4.4 Cleaning the Elevation features

In [81]:
df_lga_features['Elevation'].unique()

array(['104m (341ft)', '30m', '1500ft', '200 meters', '190m', '140m',
       '250m', '500 meters;', '1500m', '200 meters above sea level',
       '100 meters', '40m', '50m,', '100m', '400m,', '123 m (404 ft)',
       '117m', '152 meters', '167 meters', '1092m', '350m', '625 meters',
       '223m', '900m', '446m', '700m', '300m', '500m,', '143m',
       '500 meters', '500m', '625m', '160 meters', '1400m', '305m',
       '207m,', '300 meters', '40 meters', 'Sea level', '32 meters',
       '41m', '23m', '36 meters', '100 meters,',
       '3 meters above sea level', '92m', '50m', '50m above sea level',
       '200m above sea level', '20m above sea level', '50 meters', '54m',
       '10m', '92 meters', '1000m', '600m', '24 meters', '45m,', '50m;',
       '30 meteres', 'Below 100m', '200 meters,', '130m', '67m', '132m',
       '120m', '150 meters', '84m', '65m', '150m', '210m',
       '100-200 meters above sea level', '211m', '20 meters',
       '45m (148ft) above sea level', '332m', '543m',

In [82]:
# extracting height and thier units
df_lga_features['Elevation'] = df_lga_features.apply(extract_numbers_units, values=['m', 'ft'], column='Elevation', axis=1)

# replace the values of Sea level, Sea Level with 0.25m and Elevated with null
value_list =[('Sea level', '0.25m'), ('Sea Level', '0.25m'), ('Elevated', '')]
df_lga_features['Elevation'] = replace_words(df_lga_features, repls= value_list, column='Elevation')

# convert all heights to meters
df_lga_features['Elevation'] = df_lga_features.apply(convert_values, 
                     values=[('m', 1), ('ft', 0.3048)], 
                     column='Elevation', axis=1)

df_lga_features['Elevation'].unique()

array([104.0, 30.0, 457.20000000000005, 200.0, 190.0, 140.0, 250.0, 500.0,
       1500.0, 100.0, 40.0, 50.0, 400.0, 123.0, 117.0, 152.0, 167.0,
       1092.0, 350.0, 625.0, 223.0, 900.0, 446.0, 700.0, 300.0, 143.0,
       160.0, 1400.0, 305.0, 207.0, 0.0, 32.0, 41.0, 23.0, 36.0, 3.0,
       92.0, 20.0, 54.0, 10.0, 1000.0, 600.0, 24.0, 45.0, 130.0, 67.0,
       132.0, 120.0, 150.0, 84.0, 65.0, 210.0, 211.0, 332.0, 543.0, 616.0,
       450.0, 800.0, 1200.0, 423.0, 1268.0, 2500.0, 80.0, 987.0, 366.0,
       116.0, 147.0, 82.0, 1546.0, 144.0, 90.0, 124.0, 1010.0, 408.0,
       1181.0, 75.0, 27.0, 650.0, 12.0, 25.0, 115.0, 78.0, 110.0, 72.0,
       55.0, 214.0, 562.0, 630.0, 750.0, 460.0, 461.0, 462.0, 1300.0,
       458.0, 1539.0, 343.0, 280.0, 230.0, 314.0, 220.0, 357.0, 390.0,
       215.0, 173.0, 320.0, 29.0, 762.0, 273.0, 1577.0, 135.0, 39.0,
       151.0, 2.0, 6.0, 532.0, 512.0, 125.0, 192.0, 456.0, '', 56.0, 15.0,
       121.0, 156.0, 506.0, 850.0, 246.0, 425.0, 234.0, 622.0, 535.0,


#### 1.1.4.5 Cleaning the Population features

In [83]:
df_lga_features['Population'].unique()

array(['382,784', '1 million', '150,000', '100,000', '140,498', '79,231',
       '50,000;', '500,000', '200,000', '50,000', '150,000,', '120,000',
       '100,000,', '259,259', '142,478', '127,167', '125,000', '20,000',
       '160,000', '24,000', '95,000', '400,000', '156,235', '48,739',
       '254,000,', '250,000', '83,000', '450,000', '214,225', '114,000',
       '334,382', '50,000,', '142,300', '1,000', '370,000', '226,564',
       '25,000;', '551,000', '111,200', '149,630', '234,460', '85,000',
       '230,000', '187,096', '105,000', '152,000', '300,000', '214,273',
       '153,000', '6.3 Million', '80,000', '102,000,', '200,000,',
       '30,000,', '5000', '10,000', '214,000', '260,000', '140,000',
       '119,077', '3.5 million', '267,859', '1,000,000', '181,000',
       '25,000', '340,000', '170,000', '78,000', '60,000', '256,000',
       '156,555', '128,611;', '202,725', '234,117', '158,642', '218,000',
       '478,000', '126,000', '6.4 million', '100,000;', '1.2 million',
  

In [84]:
# extracting population and thier units
df_lga_features['Population'] = df_lga_features.apply(extract_numbers_units, 
                                                      values=['million', ''], 
                                                      column='Population', axis=1)

# converting all popuplation to values in thousands
df_lga_features['Population'] = df_lga_features.apply(convert_values, 
                     values=[('million', 1000000)], 
                     column='Population', axis=1)

df_lga_features['Population'].unique()

array(['382784', 1000000.0, '150000', '100000', '140498', '79231',
       '50000', '500000', '200000', '120000', '259259', '142478',
       '127167', '125000', '20000', '160000', '24000', '95000', '400000',
       '156235', '48739', '254000', '250000', '83000', '450000', '214225',
       '114000', '334382', '142300', '1000', '370000', '226564', '25000',
       '551000', '111200', '149630', '234460', '85000', '230000',
       '187096', '105000', '152000', '300000', '214273', '153000', '6.',
       '80000', '102000', '30000', '5000', '10000', '214000', '260000',
       '140000', '119077', 5000000.0, '267859', '1000000', '181000',
       '340000', '170000', '78000', '60000', '256000', '156555', '128611',
       '202725', '234117', '158642', '218000', '478000', '126000',
       4000000.0, 2000000.0, '14000', '90000', '74000', '276000', '45000',
       '234567', '23000', '82000', '257000', '75000', '394041', '50000 ',
       '156000', '185000', '176000', '183555', '183049', '183000',
      

#### 1.1.4.6 Cleaning the Major Occupation features

In [85]:
df_lga_features['Major Ocupation'].unique()

array(['Trading and Commerce', 'Trading', 'Agriculture', 'Farming',
       'Farming;', 'Farming,', 'Agriculture,', 'farming', 'Trading,',
       'Fishing', 'Trading and Service', 'Trading and Farming',
       'Trading and Manufacturing', 'Trading and Agriculture',
       'Commercial trading', 'Fishing,', 'farming;', 'Manufacturing',
       'Agriculture;', 'Commerce', 'Agriculture |',
       'Fishing and Oil and Gas Industry', nan, 'Government',
       'Farming/Agriculture', 'Trade and Services', 'Trade', 'Agriculure',
       'Fishing and tourism', 'White-collar jobs', 'Service industries',
       'Service industry', 'Trading and Business,', 'Retail;',
       'Agricultural', 'Agriculture.', 'Cement production',
       'Farming/Fishing', 'Farming and Mining;', 'Trading and commerce',
       'Oil and gas industry', 'Farming.'], dtype=object)

In [86]:
# lets replace some values with more generic values to reduce the number of categories
words_list = [
                ('Trad', 'Trading'), ('Commerc', 'Commercial'), ('Agric', 'Agriculture'),
                 ('Farm', 'Farming'), ('Fish', 'Fishery'), ('service', 'Commercial'),
                 ('Manufactur', 'Manufacturing'), ('Oil', 'Oil and Gas'), ('Government', 'Civil servant'),
                 ('Touris', 'Tourism'), ('white', 'White Collar'), ('business', 'Commercial'),
                 ('retail', 'Trading'), ('cement', 'Cement Manufacturing'), ('Mining', 'Minning')
            ]

df_lga_features['Major Occupation'] = replace_words(df_lga_features, repls=words_list, column='Major Ocupation') 
df_lga_features['Major Occupation'].unique() 

array(['trading', 'agriculture', 'farming', 'fishery', 'manufacturing',
       'commercial', '', 'civil servant', 'white collar',
       'cement manufacturing', 'oil and gas'], dtype=object)

#### 1.1.4.7 Cleaning the Political stability features

In [87]:
df_lga_features['Political stability'].unique()

array(['Relatively Stable', 'Relatively stable', 'Stable',
       'Relatively Stable;', 'Stable,', 'Fairly stable', 'Unstable',
       'relatively stable', 'Relatively Stable,', 'Stable;',
       'Fairly Stable', 'Relatively stable,', 'Moderately stable',
       'Somewhat stable', 'Fairly Stable,', 'stable;', 'Unstable,',
       'Moderate', 'Stable |', 'Relatively stable;', 'Generally Stable',
       nan, 'High', 'Generally stable', 'Stable.', 'Moderately Stable',
       'Strong', 'Fairly stable,', 'Very stable'], dtype=object)

In [88]:
# lets replace some values with more generic values to reduce the number of categories
words_list = [('generally stable', 'stable'), ('high', 'very stable'), ('strong', 'very stable'),
                   ('moderately stable', 'moderate'), ('somewhat stable', 'fairly stable')]

df_lga_features['Political stability'] = replace_words(df_lga_features, repls=words_list, column='Political stability') 
df_lga_features['Political stability'].unique() 

array(['relatively stable', 'stable', 'relatively stable;', 'stable,',
       'fairly stable', 'unstable', 'relatively stable,', 'stable;',
       'moderate', 'fairly stable,', 'unstable,', 'stable |', '',
       'very stable', 'stable.'], dtype=object)

#### 1.1.4.8 Cleaning the Economical Situation features

In [89]:
df_lga_features['Economic Situation'].value_counts()

Developing                           380
Poor                                 110
Developing,                           59
Struggling                            47
Growing                               17
Underdeveloped                        12
Average                               11
Developing;                            9
Weak                                   9
Low                                    8
Poor,                                  7
Struggling,                            5
Growing,                               5
Slowly developing                      3
Moderate                               3
Improving                              3
Booming                                2
Good                                   2
Low income                             2
Strong                                 2
Below Average                          2
Relatively poor                        2
Underdeveloped,                        2
Struggling;                            2
Mostly Agricultu

In [90]:
# lets replace some values with more generic values to reduce the number of categories
words_list = [('growing', 'developing'), ('struggling', 'underdeveloped'), ('weak', 'underdeveloped'),
                   ('low', 'underdeveloped'), ('poor', 'underdeveloped'), ('slow', 'developing'), 
                   ('below average', 'underdeveloped'), ('moderate', 'average'), ('improving', 'developing'),
                   ('booming', 'developed'), ('good', 'developed'), ('strong', 'developed'), ('agricultur', ''),
                   ('oil', ''), ('challenging', 'underdeveloped'), ('satisfactory', 'developed'), ('declining', ''), 
                  ('deteriorating', ''), ('fairly', 'developing')
            ]

df_lga_features['Economic Situation'] = replace_words(df_lga_features, repls=words_list, column='Economic Situation') 
df_lga_features['Economic Situation'].unique() 

array(['underdeveloped', 'developing', 'developing,', 'average',
       'underdeveloped;', 'underdeveloped,', '', 'developing;',
       'developed', 'developing |', 'average,', 'developing.',
       'underdeveloped.'], dtype=object)

#### 1.1.4.9 Cleaning the Average Cost of living features

In [91]:
df_lga_features['Average Cost of living'].value_counts()

â‚¦50,000                   149
â‚¦40,000                    45
â‚¦30,000                    35
N50,000                      27
â‚¦50,000,                   25
                           ... 
â‚¦30,000 ($74.57)            1
â‚¦50,000/$125 per month      1
â‚¦20,000                     1
NGN 80,000                    1
N30,000,                      1
Name: Average Cost of living, Length: 215, dtype: int64

In [92]:
currencies = [('$', 348.6), ('N', 1), ('NGN', 1)]
df_lga_features['Average Cost of living'] = df_lga_features.apply(extract_convert_curr,
                                                                  column='Average Cost of living', 
                                                                  currencies = currencies, axis=1)
df_lga_features['Average Cost of living'].unique()

array([108763.20000000001, 50000.0, 40000.0, 43575.0, 30000.0, 60000.0,
       34860.0, 63445.200000000004, 70000.0, 45000.0, 35000.0, 42877.8,
       80000.0, 100000.0, 25000.0, 53335.8, 38346.0, 31374.000000000004,
       29631.000000000004, 54730.200000000004, 75000.0, 58000.0, 41832.0,
       54033.0, 52290.0, 69720.0, 150000.0, 33117.0, 25796.4, 45318.0,
       26842.2, 61005.00000000001, 200000.0, 26145.0, 44882.25, 85000.0,
       87150.0, 20000.0, 12898.2, 65536.8, 22310.4, 25447.800000000003,
       62748.00000000001, 120000.0, 35557.200000000004, 25099.2,
       25995.102, '', 35905.8, 42529.200000000004, 53684.4,
       45666.600000000006, 21613.2, 65000.0, 104580.0, 55776.0,
       128982.00000000001, 71463.0, 41134.8, 64491.00000000001, 250.0,
       244020.00000000003, 90000.0, 67628.40000000001, 55000.0, 44620.8,
       174300.0, 40786.200000000004, 30328.2, 63096.600000000006],
      dtype=object)

#### 1.1.4.10 Cleaning the Employment rate

In [98]:
df_lga_features['Employment rate'].value_counts()

60%     113
45%      80
70%      76
40%      76
65%      49
       ... 
22%       1
35%;      1
78%,      1
82%       1
39%       1
Name: Employment rate, Length: 79, dtype: int64

In [99]:
# apply function to extract only the digits in the value as float
df_lga_features['Employment rate'] = df_lga_features.apply(get_numbers_without_units
                                                                , column='Employment rate'
                                                                , no_match_value=''
                                                                , axis=1)
df_lga_features['Employment rate'].unique()

array([65.0, 45.0, 60.0, 70.0, 40.0, 35.0, 75.0, 30.0, 55.0, 68.0, 62.0,
       85.0, 50.0, 78.0, 38.0, 49.0, 80.0, 54.0, 72.0, 47.0, 63.0, 46.0,
       12.0, 25.0, 95.0, 48.0, 94.0, 34.0, 43.0, 69.0, 52.0, 3.0, 83.0,
       89.0, 22.0, '', 90.0, 82.0, 42.0, 87.0, 74.0, 64.0, 23.0, 88.0,
       58.0, 32.0, 36.0, 73.0, 56.0, 8.0, 39.0], dtype=object)

We have completed the cleaning of the first part of the dataset. We look at the other parts

In [100]:
df_lga_features.iloc[0:, 11:].head()

,Transport infrastructure,Longitude,Latitude,Income Equality,Major ethnic group,Communication network,Infrastructure development,Dominant Age group,Average family size,Average education level,...,Has Forest,Has Mountain,Has ocean,Has valley,Has desert,Has rock,Has plateau,Has creeks,Has none,Major Occupation
0,Well-developed,7.39Â° E,5.05Â° N,Low,Igbo,Good,Developing,25-44 years,4 people,High school diploma,...,0,0,0,0,0,0,0,0,1,trading
1,Decent,7.37 E,5.53 N,Low,Igbo,Well-established,Limited,25-44,4,High school,...,0,0,0,0,0,0,0,0,1,trading
2,Poor,7.9 E,5.5 N,Low,Igbo,Limited,Slow,Youth,4,High school diploma,...,0,0,0,0,0,0,0,0,0,agriculture
3,Good,7.5Â° E,5.6Â° N,Low,Igbo,Well established,Moderately developed,25-40 years,5,Secondary school,...,0,0,0,0,0,0,0,0,0,farming
4,Poor,7.5Â°E,5.4Â°N,Low,Ibo,Limited,Developing,Youth,4,High School,...,0,0,0,0,0,0,0,0,0,farming


#### 1.1.4.11 Cleaning the Ethnic Marginalization

In [103]:
df_lga_features['Ethnic Marginalization'].value_counts()

Minimal                                                  272
None                                                     106
High                                                      64
Low                                                       56
Moderate                                                  39
None,                                                     30
Minimal,                                                  27
Marginalized                                              19
Significant                                               17
Yes                                                       11
Minorities                                                 9
Marginalized,                                              7
Minimal;                                                   7
Some marginalization                                       6
Low,                                                       6
Not significant                                            4
None;                   

In [105]:
# lets replace some values with more generic values to reduce the number of categories
words_list = [('Low', 'Minimal'), ('High', 'Significant'), ('Moderate', 'Minimal'), ('marginaliz', 'Significant'),
                   ('yes', 'significant'), ('Minorities', 'Significant'), ('Minor', 'Minimal'), ('Negligible', 'Minimal'),
                   ('Not signifigant', 'Minimal'), ('significant', 'significant'), ('some', 'minimal'), ('Mild', 'Minimal'),
                   ('limited', 'Minimal'), ('severe', 'significant'), ('present', 'significant'), ('inclusive', 'none'), 
                   ('common', 'significant'), ('experienced', ''), ('minimal', 'minimal'), ('none', 'none')
            ]

df_lga_features['Ethnic Marginalization'] = replace_words(df_lga_features, repls=words_list, column='Ethnic Marginalization') 
df_lga_features['Ethnic Marginalization'].unique() 

array(['minimal', 'none', 'significant', ''], dtype=object)

#### 1.1.4.12 Cleaning the Transport Infrastructure

In [102]:
df_lga_features['Transport infrastructure'].value_counts()

Poor                                          224
Limited                                       179
Good                                          117
Good,                                          31
Well-developed                                 29
Poor,                                          25
Limited,                                       12
Basic                                          11
Adequate                                       10
Underdeveloped                                  9
Fair                                            6
Well-developed,                                 6
Good road network                               6
Decent                                          6
Poor;                                           5
Moderate                                        5
Developing                                      5
Fairly Developed                                3
Good;                                           3
Average                                         3


#### 1.1.4.13 Cleaning the Longitude

In [115]:
df_lga_features['Longitude'].unique()

array(['7.39Â° E', '7.37 E', '7.9 E', '7.5Â° E', '7.5Â°E', '7.45',
       '7.5440Â° E', '7.62Â° E;', '7.52Â°E', '6.9 E', '8Â°E,', '6.8Â°E',
       '7.5Â°E,', '7.8936Â° E', '7.3824Â° E', '12.4219Â° E', '12.5Â°E',
       '11.9972Â° E', '9.06Â°E', '12.4353Â° E', '12.10 E', '6Â°E',
       '12.9960Â° E', '10.3706Â° E', '13.1725Â° E', '12.7599Â° E,',
       '12.3Â°E', '12.3958Â° E', '13.2354Â° N', '13.2517Â° N', '10.45 E',
       '12.83Â°E', '6.1Â°E', '12.4656Â° E', '12.5Â° E,', '12.4Â° E',
       '7.9267Â° E', '7.9833Â° E', '7.8243Â° E', '7.8321Â° E', '7.802Â°E',
       '7.8341Â° E', '7.7000Â° E', '7.8 E', '7.5043Â° E', '7.6452Â° E',
       '6.6', '7.57E,', '7.8Â° E', '7.7285Â° E', '3.5Â° E', '7.34Â°E',
       '6.5Â° E', '7.7833Â° E', '7.7649Â° E', '7.9340Â° E', '7.9354Â° E',
       '5.09Â° E', '8.5Â° E', '7.23Â°E', '8.6724Â° E', '7.5692Â° E,',
       '8.4;', '7.8523Â° E', "7Â°30'N", '7.5464Â° E', '7.9290Â° E',
       '6.02Â° E', '6.0494Â° E', '6.4Â°E', '6.8Â° E,', '6.16Â° E',
       '7.06Â

In [124]:
df_lga_features['Longitude'] = df_lga_features.apply(get_numbers_without_units, column='Longitude', no_match_value='', axis=1)
df_lga_features['Longitude'].unique()[0:20]

array(['7.39', '7.37', '7.9', '7.5', '7.45', '7.5440', '7.62', '7.52',
       '6.9', '8', '6.8', '7.8936', '7.3824', '12.4219', '12.5',
       '11.9972', '9.06', '12.4353', '12.10', '6'], dtype=object)

#### 1.1.4.14 Cleaning the Latitude

In [125]:
df_lga_features['Latitude'].unique()

array(['5.05Â° N', '5.53 N', '5.5 N', '5.6Â° N', '5.4Â°N', '5.22',
       '5.0588Â° N', '5.76Â° N;', '4.8Â°N', '5.45Â°N', '5.8Â°N', '5.2 N',
       '6Â°N,', '5.7Â°N', '5.5Â°N,', '5.5320Â° N', '5.5336Â° N',
       '9.9231Â° N', '9.2Â°N', '9.2511Â° N', '8.98Â°N', '9.3118Â° N',
       '9.20 N', '8Â°N', '8.8482Â° N', '9.8480Â° N', '10.8963Â° N',
       '9.0359Â° N,', '9.8Â°N', '10.5966Â° N', '12.5122Â° E',
       '12.7288Â° E', '9.85 N', '9.79Â°N', '9.9Â°N', '9.1342Â° N',
       '9.2Â° N,', '9.3Â° N', '4.9689Â° N', '4.9833Â° N', '4.6420Â° N',
       '4.6236Â° N', '5.024Â°N', '5.0073Â° N', '4.5833Â° N', '4.7 N',
       '4.7218Â° N', '4.7017Â° N', '5.5', '5.00N,', '4.6Â° N',
       '5.1750Â° N', '7.5Â° N', '4.75Â°N', '4.8Â° N', '5.0167Â° N',
       '4.6943Â° N', '4.8015Â° N', '4.7580Â° N', '5.12Â° N', '5Â° N',
       '5.14Â°N', '4.8181Â° N', '4.9496Â° N,', '4.9;', '4.7674Â° N',
       "5Â°05'E", '4.7434Â° N', '5.0330Â° N', '6.07Â° N', '6.1704Â° N',
       '6.1Â°N', '6.10Â° N,', '6.21Â° N', '

In [126]:
df_lga_features['Latitude'] = df_lga_features.apply(get_numbers_without_units, column='Latitude', no_match_value='', axis=1)
df_lga_features['Latitude'].unique()[0:20]

array(['5.05', '5.53', '5.5', '5.6', '5.4', '5.22', '5.0588', '5.76',
       '4.8', '5.45', '5.8', '5.2', '6', '5.7', '5.5320', '5.5336',
       '9.9231', '9.2', '9.2511', '8.98'], dtype=object)

#### 1.1.4.15 Cleaning the Income Equality

In [128]:
df_lga_features["Income Equality"].value_counts()

Low                       325
Unequal                   190
Moderate                   53
Unequal,                   41
Low,                       20
Moderate,                  16
Poor                       13
High                       11
High inequality             9
Unequal;                    7
Moderately unequal          5
Fair                        3
High income inequality      3
Low;                        3
Relatively equal            2
Below average               2
Moderate;                   2
Unequal.                    2
Moderately Unequal          2
Highly unequal,             1
Fair;                       1
Average                     1
High Income Inequality      1
Very Unequal                1
Relatively Unequal          1
Highly Unequal              1
Fair,                       1
Low Equality                1
Medium                      1
Relatively Unequal,         1
Relatively unequal          1
unequal;                    1
Very unequal                1
Unequal | 

In [129]:
words_list = [('Low', 'Low'), ('Moderate', 'Moderate'), ('High', 'High'), ('equal', 'Moderate'), ('Unequal', 'high'),
                   ('Poor', 'High'), ('Unbalance', 'Unequal'), ('fair', 'Moderate'), ("medium", 'Moderate'),
                   ('Very unequal', 'High'), ('Below Average', 'Unequal'), ('Average', 'Moderate')
                  ]

df_lga_features['Income Equality'] = replace_words(df_lga_features, repls=words_list, column='Income Equality') 
df_lga_features['Income Equality'].unique() 

array(['low', 'moderate', 'high', 'unequal', ''], dtype=object)

#### 1.1.4.16 Cleaning the Communication network features

In [131]:
df_lga_features['Communication network'].value_counts()

Limited                                          422
Good                                              77
Limited,                                          43
Adequate                                          27
Strong                                            22
Well-established                                  17
Well-developed                                    14
Good,                                             12
Well-connected                                     9
Strong,                                            8
Limited;                                           8
Adequate,                                          5
Reliable                                           5
Limited but improving                              4
Average                                            3
Well-established,                                  3
Good;                                              3
Basic                                              3
Well-connected,                               

In [139]:
words_list = [('Fairly', 'Good'), ('Well', 'Strong'), ('Reliable', 'Strong'), ('limited', 'limited'), 
                   ('basic', 'limited'), ('High', 'Strong'), ('Average', 'Good'), ('poor', 'limited'),
                   ('advanced', 'strong'), ('good', 'good'), ('broad', 'Strong'), ('comprehensive', 'strong'),
                   ('sparse', 'Limited'), ('Medium', 'Adequate'), ('Moderate', 'Adequate'), ('available', 'good'),
                   ('restricted', 'limited'), ('patchy', 'limited'), ('internet and telephone', ""), ('accessible', 'good'),
                   ('fair', 'good'), ('adeq', 'adequate'), ('stro', 'strong')
                  ]

df_lga_features['Communication network'] = replace_words(df_lga_features, repls=words_list, column='Communication network') 
df_lga_features['Communication network'].unique() 

array(['good', 'strong', 'limited', 'adequate', ''], dtype=object)

#### 1.1.4.17 Cleaning the Infrastructure development feature

In [134]:
df_lga_features["Infrastructure development"].value_counts()

Developing                  244
Underdeveloped               80
Limited                      79
Poor                         31
Basic                        30
                           ... 
Ongoing projects,             1
Infrastructural deficits      1
Slow;                         1
Well-established              1
Adequate;                     1
Name: Infrastructure development, Length: 66, dtype: int64

In [136]:
words_list = [('Underdeveloped', 'Poor'), ('Moderate', 'Moderate'), ('Fair', 'Fairly Developed'), ('limited', 'limited'), 
                   ('basic', 'limited'), ('Slow', 'limited'), ('Average', 'Good'), ('improv', 'developing'), 
                   ('minimal', 'limited'), ('high', 'adequate'), ('good', 'adequate'),
                   ('well', 'adequate'), ('lack', 'poor'), ('grow', 'developing'), ('developed', 'adequate'),
                   ('progress', 'developing'), ('low', 'poor'), ('going', 'developing'), ('expanding', 'developing'),
                   ('deficits', 'poor'), ('neglected', 'poor'), ('subpar', 'poor'), ('inadequate', 'poor'), 
                    ('devel', 'developing'), ('poor', 'poor'), ('adeq', 'adequate')
                  ]


df_lga_features['Infrastructure development'] = replace_words(df_lga_features, repls=words_list, column='Infrastructure development') 
df_lga_features['Infrastructure development'].unique() 

array(['developing', 'limited', 'moderate', 'adequate', 'poor', ''],
      dtype=object)

#### 1.1.4.18 Cleaning the Infrastructure development feature

In [140]:
df_lga_features["Dominant Age group"].unique()

array(['25-44 years', '25-44', 'Youth', '25-40 years', 'Youth (below 25)',
       '20-40', '18-35 years;', 'Young adults', '25-45 years', '25-40',
       'Youth,', '25-44,', '18-35 years', 'Youth (18-30)', '18-35',
       '20-40 years', 'Youth (18-35)', '20-35 years old', '20-35 years',
       '21-40 years', '25-34', '21-35 years', '25-34 years,', '25-45',
       '25-44 age group', '20-35', 'Youthful,', '25-40 years old',
       '25-35 years', '25-35', '25-40;', '20-39', 'Youth (under 35)',
       'Young Adults,', 'Youthful', 'Young adults (20-39)', '30-45 years',
       '16-30 years', '15-64 years', '25-40,', '20-35,', '20-35 years,',
       'Youth (18-35 years)', '18-45 years', 'Young adults (18-35)',
       '25-40 yrs', 'Youth (15-35)', '27-35', 'Youth (below 35)',
       'Youth (15-29yo)', '20-30 years', '20-40 years old', '20-30,',
       'Youth (under 25)', 'youth;', '25-54 years', '25-44 years old',
       '15-35 years', '15-44 years', '20-40 years,', '25-40 years;',
       '25-

In [156]:
df_lga_features["Dominant Age group"] = df_lga_features.apply(extract_age_range, column="Dominant Age group", axis=1)
df_lga_features["Dominant Age group"]

0      25-44
1      25-44
2      15-35
3      25-40
4      15-35
       ...  
726    25-40
727    15-35
728       30
729    25-34
730    25-40
Name: Dominant Age group, Length: 731, dtype: object

#### 1.1.4.19 Cleaning the Average family size features

In [158]:
df_lga_features["Average family size"].value_counts()

4                     287
5                     215
6                      44
5,                     33
4,                     32
4 persons              16
5 persons              14
4 members              10
5 people               10
5 members              10
5;                      8
4 people                8
4;                      8
6,                      7
6 people                4
7                       3
6 persons               3
4 persons,              2
4-5 people              2
Primary education.      1
5 people,               1
Five,                   1
6 members,              1
4 people,               1
Secondary school.       1
4-6 people              1
4-5                     1
3-4 people              1
4-6                     1
6 persons,              1
4 |                     1
6 members               1
8,                      1
Name: Average family size, dtype: int64

In [159]:
# The extract age range function can be used on this field
df_lga_features["Average family size"] = df_lga_features.apply(extract_age_range, column="Average family size", axis=1)
df_lga_features["Average family size"]

0      4
1      4
2      4
3      5
4      4
      ..
726    6
727    4
728    6
729    6
730    4
Name: Average family size, Length: 731, dtype: object

#### 1.1.4.20 Cleaning the Average Education Level

In [160]:
#### Checking Average Education Level
df_lga_features["Average education level"].value_counts()

Secondary school           198
Secondary School           108
High school                 87
High school diploma         35
High School                 35
                          ... 
College degree               1
Secondary school level,      1
High School Diploma,         1
University Degree            1
45,000 Naira.                1
Name: Average education level, Length: 74, dtype: int64

In [165]:
words_list = [('Secondary', 'Secondary School'), ('High', 'Secondary School'), ('Bachelor', 'Bachelor Degree'),  
                   ('Primary', 'Primary School'), ('Diploma', 'Diploma'), ('college', 'Diploma'), 
                   ('university', 'Bachelor Degree'), ('basic', 'primary school'), ('tertiary', 'Bachelor Degree'),
                   ('middle', 'secondary'), ('naira', ''), ('100', '')
                   
             ]

df_lga_features['Average education level'] = replace_words(df_lga_features, repls=words_list, column='Average education level')
df_lga_features['Average education level'].unique() 

array(['secondary school', 'bachelor degree', 'primary school', 'diploma',
       ''], dtype=object)

#### 1.1.4.21 Cleaning the Average Income in Naira

In [168]:
df_lga_features["Average Income in Naira"].value_counts()

â‚¦50,000             47
â‚¦40,000             44
â‚¦80,000             38
â‚¦70,000             35
â‚¦100,000            30
                      ..
N45,                   1
â‚¦35,000 ($88)        1
â‚¦30,000 ($76)        1
â‚¦35,000 ($86.84)     1
N70,                   1
Name: Average Income in Naira, Length: 262, dtype: int64

In [169]:
currencies = [('$', 348.6), ('N', 1), ('NGN', 1)]
df_lga_features['Average Income in Naira'] = df_lga_features.apply(extract_convert_curr,
                                                                  column='Average Income in Naira', 
                                                                  currencies = currencies, axis=1)
df_lga_features['Average Income in Naira'].unique()

array([135954.0, 40000.0, 50000.0, 80000.0, 52290.0, 62000.0, 40.0,
       150000.0, 69720.0, 100.0, 100000.0, 75000.0, 45318.0, 35000.0,
       30000.0, 104580.0, 25000.0, 34511.4, 60000.0, 45.0, 70000.0,
       62399.4, 70.0, 30328.2, 350000.0, 25796.4, 120000.0, 55776.0,
       300.0, 63793.8, 200000.0, 68674.20000000001, 72857.40000000001,
       45000.0, 280000.0, 59262.00000000001, 51000.0, 22310.4, 75.0, 30.0,
       26145.0, 200.0, 150.0, 250.0, 99351.0, 51592.8, 500000.0, 300000.0,
       27190.800000000003, 20000.0, 41832.0, 29631.000000000004,
       127284.31800000001, 43575.0, 120.0, 65000.0, 60.0,
       62748.00000000001, 71804.628, 261450.00000000003, 34860.0,
       61005.00000000001, 85000.0, 50895.600000000006, 110400.0, 50.0,
       250000.0, 20.0, 35.0, 180000.0, 21613.2, 52638.600000000006,
       17778.600000000002, 42529.200000000004, 33465.600000000006,
       25447.800000000003, 30676.800000000003, 26493.600000000002,
       30272.424000000003, '', 111552.0, 5

## 1.2 Data integration and Transfromation

Data Integration involves combining data from multiple sources into a single dataset. Data transformation involves converting data into a more suitable format for analysis, such as normalizing or scaling data.

The ACLED and the GTD dataset were collected by different organisations with different objectives. Both datasets must have been collected to suit the usecase of the different institutions. Hence, the explanation for the wide disparity in the number of features. While the GTD collected more features, almost three folds of the features collected by ACLED, they do not cover as much events as the ACLED datasets. There is also a need to integrate datasets containing the records of attacks with the Local government areas of the attacks dataset and subsequently the dataset containing selected features of the LGA.  

Hence, this section of the project seeks to preprocess all the datasets to cummulatively suit the purpose of this project.

### 1.2.1 Exploring Mergability of the Datasets

The ACLED and the lga, LGA features datasets can all the merged. On the ACLED dataset, there is a admin2 field which contains the name of the LGA where each event occur. The LGA and the LGA feature dataset contains name fields which allows the three datasets to be merged using the LGA name.

Attempting to bring the GTD dataset into the mix is problematic because the closest it has in common to the LGA name is the location field. Above 50% of this field is empty. The closest it also has to the ACLED dataset is the date field. This field cannot be used to reliably merge both datasets. Therefore, the GTD dataset will be used indepependently should its need arise in this project.

The primary focus for this project will be the ACLED, the LGA dataset and the LGA feature datasets.

In [93]:
# Attempting to merge the ACLED dataset with the LGA features
df_acled = df_acled.rename(columns={'admin2': 'lga_name'}) # rename the admin2 to lga_name in acled
df_lga_features = df_lga_features.rename(columns={'Lga Name': 'lga_name'}) # rename the Lga Name to lga_name in LGA features dataset
merged_df = pd.merge(df_acled, df_lga_features, on='lga_name')

In [94]:
merged_df.head()

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,inter1,actor2,...,Has Forest,Has Mountain,Has ocean,Has valley,Has desert,Has rock,Has plateau,Has creeks,Has none,Major Occupation
0,NIG30571,2023-03-24,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Nigeria),6,unknown,...,0,0,0,0,0,0,0,0,0,agriculture
1,NIG30686,2023-03-23,2023,1,Political violence,Violence against civilians,Abduction/forced disappearance,Fulani Ethnic Militia (Nigeria),4,Civilians (Nigeria),...,0,0,0,0,0,0,0,0,0,agriculture
2,NIG30637,2023-03-22,2023,1,Political violence,Violence against civilians,Attack,Fulani Ethnic Militia (Nigeria),4,Civilians (Nigeria),...,0,0,0,0,0,0,0,0,0,agriculture
3,NIG29466,2023-01-04,2023,1,Political violence,Battles,Armed clash,Hausa Ethnic Militia (Nigeria),4,Fulani Ethnic Militia (Nigeria),...,0,0,0,0,0,0,0,0,0,agriculture
4,NIG29455,2023-01-03,2023,1,Political violence,Battles,Armed clash,Hausa Ethnic Militia (Nigeria),4,Fulani Ethnic Militia (Nigeria),...,0,0,0,0,0,0,0,0,0,agriculture


Hence, it is confirmed the both dataset can be merged without underlying issues.

## 1.3 Exporting the Preprocessed Datasets

In [173]:
#save dateset into the cleaned folder
df_acled.to_csv('datasets/preprocessed/df_acled.csv', index=False)

# save this dataset into the cleaned folder
df_lga.to_csv('datasets/preprocessed/df_lga.csv', index=False)

# save dataset as a new csv file
df_gtd.to_csv('datasets/preprocessed/df_gtd.csv', index=False)

# save this dataset into the cleaned folder
df_lga_features.to_csv('datasets/preprocessed/df_lga_features.csv', index=False)